In [1]:
# 구글 드라이브 사용 권한 설정
import os 
from google.colab import drive
drive.mount('/gdrive')
data_path = '/gdrive/MyDrive/Colab Notebooks/Capstone/'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
%cd /gdrive/MyDrive/Colab Notebooks/Capstone/

/gdrive/MyDrive/Colab Notebooks/Capstone


In [3]:
import pandas as pd
import cv2
import numpy as np
import ast
from matplotlib import pyplot as plt

In [22]:
df1 = pd.read_csv(data_path+"labeling/CSV_files/labeling_1.csv",sep=r",(?![^[]*\])",engine="python") ## 써진 형식이 달라서 구분자 추가 
df2 = pd.read_csv(data_path+"labeling/CSV_files/labeling_2.csv",sep=r",(?![^[]*\])",engine="python")
df3 = pd.read_csv(data_path+"labeling/CSV_files/labeling_3.csv",sep=r",(?![^[]*\])",engine="python",error_bad_lines=False)
df4 = pd.read_csv(data_path+"labeling/CSV_files/labeling_4.csv")
# df5 = pd.read_csv(data_path+"labeling/CSV_files/labeling_5.csv",sep=r",(?![^[]*\])",engine="python")

<ipython-input-22-a89888639317>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df3 = pd.read_csv(data_path+"labeling/CSV_files/labeling_3.csv",sep=r",(?![^[]*\])",engine="python",error_bad_lines=False)
Skipping line 5: Expected 10 fields in line 5, saw 223. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 6: Expected 10 fields in line 6, saw 231. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 8: Expected 10 fields in line 8, saw 223. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 9: Expected 10 fields in line 9, saw 15. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 10: Expected 10 fields in line 10, saw 71. Error could possibly be due to quotes being ignored when a multi-

In [23]:
df = pd.concat([df1,df2,df3,df4], ignore_index=True) #각각 읽은 df 합치기
print(len(df1))
print(len(df2))
print(len(df3))
print(len(df4))
# print(len(df5))
# print(df5.isnull().sum())
df['ocr'][50]

24
17
10
40


'/data/upload/2/c075d03e-photo_3_2023-04-09_00-29-36.jpg'

In [30]:
### 결측 확인 
label = df['transcription']
bbox = df['bbox']
label_img = df['ocr']

print(label.isnull().sum())
print(label_img.isnull().sum())
print(bbox.isnull().sum())

0
0
0


In [31]:
label

0     ["1", "9", "0", "4", "2", "5", "8", "8", "2", ...
1     ["4", "7", "1", "9", "5", "0", "9", "1", "1", ...
2     ["8", "7", "4", "3", "1", "0", "3", "0", "7", ...
3     ["4", "7", "3", "5", "9", "2", "0", "0", "7", ...
4     ["4", "3", "2", "9", "4", "2", "1", "1", "0", ...
                            ...                        
86    ["1", "6", "8", "0", "7", "2", "2", "1", "0", ...
87    ["1", "2", "5", "8", "2", "0", "4", "0", "5", ...
88    ["2", "1", "5", "2", "1", "1", "2", "0", "2", ...
89    ["3", "5", "2", "9", "2", "2", "0", "6", "2", ...
90    ["4", "1", "2", "3", "2", "0", "0", "8", "0", ...
Name: transcription, Length: 91, dtype: object

In [32]:
# 저장된 원본 이미지 경로 다시 설정 
label_img = [path.replace(label_img[i], 
                          '/gdrive/My Drive/Colab Notebooks/Capstone/labeling/images/'+str(path.split('/')[-1])) for i,path in enumerate(label_img)]


In [34]:
### image crop
f = open("./gt_crop.txt", "w")

for n in range(len(label_img)):
  org_img = cv2.imread(label_img[n])
  box = bbox[n]
  for i,img in enumerate(eval(box)) :
      x = round(img.get('x')*float(img.get('original_width'))/100.0)
      y = round(img.get('y')*float(img.get('original_height'))/100.0)
      w = round(img.get('width')*float(img.get('original_width'))/100.0)
      h = round(img.get('height')*float(img.get('original_height'))/100.0)
      
      crop = org_img[y:y+h, x:x+w]
      crop_path = f'./cropimg/{n}_crop_{x}_{y}_{w}_{h}.jpg'
      cv2.imwrite(crop_path, crop)
      f.write(crop_path.lstrip('./')+'\t'+ str(eval(label[n])[i])+'\n')
              #/파일이름  정답 
  print(label_img[n].split('/')[-1]+str(n)+'..........DONE')
f.close()

15f6c072-KakaoTalk_Photo_2023-03-16-23-10-02_001.jpeg0..........DONE
ece4c854-KakaoTalk_Photo_2023-03-16-23-10-02_002.jpeg1..........DONE
ed4102ca-KakaoTalk_Photo_2023-03-16-23-10-02_003.jpeg2..........DONE
c3b0faab-KakaoTalk_Photo_2023-03-16-23-10-03_004.jpeg3..........DONE
96be5f3d-KakaoTalk_Photo_2023-03-16-23-10-03_005.jpeg4..........DONE
452016e1-KakaoTalk_Photo_2023-03-16-23-10-03_006.jpeg5..........DONE
851966ea-KakaoTalk_Photo_2023-03-16-23-10-03_007.jpeg6..........DONE
f6c03168-KakaoTalk_Photo_2023-03-16-23-10-03_008.jpeg7..........DONE
8ebc7a24-KakaoTalk_Photo_2023-03-16-23-10-03_009.jpeg8..........DONE
10a5b339-KakaoTalk_Photo_2023-03-16-23-10-04_010.jpeg9..........DONE
d4ef86e5-KakaoTalk_Photo_2023-03-16-23-10-04_011.jpeg10..........DONE
696d3444-KakaoTalk_Photo_2023-03-16-23-10-04_012.jpeg11..........DONE
37642b01-KakaoTalk_Photo_2023-03-16-23-10-04_013.jpeg12..........DONE
5707eaaf-KakaoTalk_Photo_2023-03-16-23-10-04_014.jpeg13..........DONE
53ecf0d1-KakaoTalk_Photo_2023-